In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import json
import sys
import re
import os
from pathlib import Path
from itertools import chain
from functools import partial
from io import StringIO

import blpapi
# import clarion
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from plotly import graph_objects as go, express as px
# from clarion import positions

from api.gui.base import View, Tabs, Store
from api.gui.market_monitor import (
    CurvesViewer, 
    BBGDownload, 
    CurveOverride, 
    Pivot, 
    BondsViewer,
    get_issuers_curves,
    get_issuer_curve
)
from api.gui.max_diff import LowestFieldCurve
from api.blpw import BlpQuery
from api.bbg import BBGField, BBGOverrides
from api.data.base import data_path, tables
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.data.processing import (
    apply_function, 
    add_years_to_maturity, 
    add_maturity_bucket,
    add_custom_rating,
    add_rating_bucket,
    add_geo_area
)

In [3]:
dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

In [4]:
bq = BlpQuery(timeout=50000).start()

In [5]:
bond_ref = add_maturity_bucket(
    add_years_to_maturity(tables.bonds_reference),
    maturity_bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 60]
)
bond_ref = add_rating_bucket(add_custom_rating(bond_ref))
bond_ref = add_geo_area(bond_ref, more_cols={'name': 'country_name'})

In [6]:
store = Store()
store.data['bond_ref'] = copy.deepcopy(bond_ref)
# store.data['issuers_curves'] = copy.deepcopy(issuers_curves)

In [ ]:
app = Tabs(
    {
        'bonds': BondsViewer(build=False),
        'curves': CurvesViewer(build=False),
        # 'overrides': CurveOverride(build=False),
        'bbg': BBGDownload(build=False),
        'pivot': Pivot(build=False),
        'max diff': LowestFieldCurve(build=False)
    },
    store=store
)
app

In [ ]:
currency = 'USD'
maturity = 30
threshold = 8
field = 'z_sprd_mid'
name = "z_spread"

In [ ]:
ic = get_issuers_curves(bond_ref=bond_ref, maturities=[maturity], thresholds={maturity: threshold})

In [43]:
def get_interval(interval, field, name=None, offset="12m"):
    return {
        "field": interval if 'interval' in interval else f"interval_{interval}",
        "name": name or f"{field}_{interval.lower().replace('interval_', '')}",
        "overrides": [
            ("calc_interval", offset),
            ("market_data_override", field)
        ]
    }

class Visual(View):
    
    _fields_mapping = {
        "px_last": "last_price",
        "yld_ytm_mid": "yield_to_maturity",
        "z_sprd_mid": "z_spread",

    }
    
    intervals = [
        'high',
        'low',
        'avg'
    ]
    
    rating_thresholds = ['BBB', 'BB', 'B']
    rating_names = ['IG', 'BB', 'B', 'Distr']
    rating_colors = ['black', 'blue', 'grey', 'green']
    rating_opacity = [1, -1, 0, 2]
    rating_opacity = [0.9, 0.7, 0.5, 0.3]
    
    
    def __init__(self, bond_ref=None, **kwargs):
        self._bond_ref = bond_ref
        super().__init__(**kwargs)
    
    def make_widgets(self, **kwargs):
        curr_sel = v.Autocomplete(
            v_model='USD',
            items=[*self.bond_ref['currency'].drop_duplicates()],
            label='currency',
            dense=True,
            outlined=True
        )
        
        maturity_tf = v.TextField(
            v_model=30,
            label='maturity',
            clearable=True,
            dense=True,
            outlined=True,
            type="number",

        )
        
        
        offset_tf = v.TextField(
            v_model='12m',
            label='offset',
            clearable=True,
            dense=True,
            outlined=True,
        )

        threshold_tf = v.TextField(
            v_model=8,
            label='threshold',
            clearable=True,
            dense=True,
            outlined=True,
            type="number",
        )
        
        pcs_tf = v.TextField(
            v_model='BGN',
            label='pcs',
            clearable=True,
            dense=True,
            outlined=True,

        )

        sep_tf = v.TextField(
            v_model='@',
            label='sep',
            clearable=True,
            dense=True,
            outlined=True,
            disabled=True
        )

        field_ac = v.Autocomplete(
            v_model='z_sprd_mid',
            items=['z_sprd_mid'],
            label='field',
            dense=True,
            outlined=True
        )
        
        dl_btn = v.Btn(
            fab=True,
            # dark=True,
            small=True,
            plain=True,
            class_='mx-2',
            children=[v.Icon(children=['mdi-download'])]
        )

        table_btn = v.Btn(
            fab=True,
            small=True,
            plain=True,
            left=True,
            children=[v.Icon(children=['mdi-poll'])]
        )
        
        self.curr_sel = curr_sel
        self.maturity_tf = maturity_tf
        self.threshold_tf = threshold_tf
        self.pcs_tf = pcs_tf
        self.sep_tf = sep_tf
        self.field_ac = field_ac
        self.offset_tf = offset_tf
        self.dl_btn = dl_btn
        self.table_btn = table_btn
        self.out = w.Output()
        
    def make_view(self, **kwargs):
        
        curr_sel = self.curr_sel
        maturity_tf = self.maturity_tf
        threshold_tf = self.threshold_tf
        pcs_tf = self.pcs_tf
        sep_tf = self.sep_tf
        field_ac = self.field_ac
        offset_tf = self.offset_tf
        dl_btn = self.dl_btn
        table_btn = self.table_btn
        out = self.out
        
        
        param_box = v.Container(
            children=[
                v.Row(
                    children=[
                        v.Col(
                            cols=2,
                            children=[curr_sel],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[maturity_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[threshold_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[offset_tf],
                            class_="my-0 py-0"
                        ),
                    ]
                ),
                v.Row(
                    children=[
                        v.Col(
                            cols=2,
                            children=[pcs_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[sep_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[field_ac],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=1,
                            children=[dl_btn, table_btn],
                            class_="my-0 py-0"
                        ),
                    ]
                ),
            ]
        )
        view = w.VBox(
            children=[
                param_box,
                out,
            ]
        )

        self.param_box = param_box
        self.view = view
    
    @property
    def bond_ref(self):
        return self.store.data['bond_ref'] if self._bond_ref is None else self._bond_ref
    
    
    def get_curves(self):
        maturity = float(self.maturity_tf.v_model)
        threshold = float(self.threshold_tf.v_model)
        self.ic = get_issuers_curves(bond_ref=bond_ref, maturities=[maturity], thresholds={maturity: threshold})
        return self.ic
    
    def get_visual_fields(self):
        field = self.field_ac.v_model
        offset = self.offset_tf.v_model
        visual_fields = [
            *[{"field": field, "name": value} for field, value in self._fields_mapping.items()],
            *[get_interval(interval, field, interval, offset) for interval in self.intervals]
        ]
        return visual_fields
    
    def download(self):
        
        
        ic = self.get_curves()
        visual_fields = self.get_visual_fields()
        securities = ic["security"].drop_duplicates().to_list()
        pcs = self.pcs_tf.v_model
        sep = self.sep_tf.v_model
        
        data = bq.bdp(
            securities=securities,
            fields=[field["field"] for field in visual_fields],
            overrides=visual_fields[-1]["overrides"],
            pcs=pcs,
            sep=sep
        ).rename({field['field']: field['name'] for field in visual_fields}, axis=1)
        
        self.mkt = data
        return data
    
    def get_chart_df(self):
        b_mkt = self.mkt
        ic = self.ic
        relevant_fields = ['ticker', 'security', 'name', 'custom_rating', 'rating_bucket', 'years_to_workout', 'maturity_bucket']
        b_mkt = ic[relevant_fields].merge(b_mkt, on='security', how='left').dropna()

        rating_thresholds = self.rating_thresholds
        rating_names = self.rating_names
        # rating_colors = self.rating_colors
        # rating_opacity = self.rating_opacity
        # rating_opacity = self.rating_opacity
        
        rating_thresholds = pd.Categorical(rating_thresholds, ordered=True, categories=tables.rating['group'].drop_duplicates()).sort_values()
        r = b_mkt['rating_bucket']
        bins = [r.cat.categories.get_loc(t) for t in rating_thresholds]
        grouper = pd.Series(np.digitize(r.cat.codes, bins, right=True)).replace({i: name for i, name in enumerate(rating_names)})
        grouper = pd.Series(pd.Categorical(grouper, ordered=True, categories=rating_names), index=r.index, name='grouper')
        b_mkt['grouper'] = grouper
        b_mkt = b_mkt.sort_values(['grouper', 'z_spread'], ascending=[False, False]).reset_index(drop=True)
        
        for col in b_mkt:
            if b_mkt[col].dtype == object:
                try:
                    b_mkt[col] = b_mkt[col].astype(float)
                except ValueError:
                    pass        
        self.b_mkt = b_mkt
        return b_mkt
    
    def get_figure(self):
        b_mkt = self.b_mkt
        width = 0.4

        fig_data = []
        lasts = []
        groups = []
        gby = b_mkt.groupby(b_mkt['grouper'].astype(object), sort=False)
        for j, (index, sub) in enumerate(gby):
            col = 'black'
            gd = [
                go.Bar(
                    name=index if j + 1 < len(gby) else 'range', 
                    x=sub['name'], 
                    y=sub['high'] - sub['low'],
                    base=sub['low'],
                    marker_color='grey',
                    opacity=0.3,
                    width=width,
                    showlegend=False if j + 1 < len(gby) else True,
                )
            ]
            fig_data.extend(gd)
            lasts.append(len(sub))
            groups.append(index)
            
        size = 7
        lw = 0.3
        lc = 'red'
        current = go.Scatter(
            name='current',
            x=b_mkt['name'],
            y=b_mkt['z_spread'],
            mode='markers',
            marker={
                'color': px.colors.qualitative.G10[0],
                'symbol': 'circle',
                'size': size,
                'line': {
                    'color': lc,
                    'width': lw
                }
            }
        )

        avg = go.Scatter(
            name='avg',
            x=b_mkt['name'],
            y=b_mkt['avg'],
            mode='markers',
            marker={
                'color': px.colors.qualitative.Plotly[5],
                'size': size,
                'symbol': 'triangle-up',
                'line': {
                    'color': lc,
                    'width': lw
                }
            }
        )
        
        fig = go.Figure(data=[
            *fig_data,
            current,
            avg
        ]).update_layout(
            barmode='stack', 
            height=1000,
            template='plotly_white',
        ).update_xaxes(
            tickangle=-45, 
            tickfont=dict(
                # family='Rockwell', 
                # color='crimson', 
                size=10
            )
        )

        lasts_ = np.cumsum(lasts) - 0.5
        for i, (last, last_, text) in enumerate(zip(lasts_, lasts, groups)):
            fig.add_vline(
                x=last, 
                opacity=0.3 if i + 1 < len(lasts_) else 0,
                annotation_text=text, 
                annotation_position="top left",
                line_dash='dot',
                annotation_xshift=(-last_ * 20) / 2 + 0.25,
                annotation_font={'size': 21}
            )

        fig.update_layout(
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )
        
        self.fig = fig
        return fig

    def link(self, **kwargs):
                
        curr_sel = self.curr_sel
        maturity_tf = self.maturity_tf
        threshold_tf = self.threshold_tf
        pcs_tf = self.pcs_tf
        sep_tf = self.sep_tf
        field_ac = self.field_ac
        offset_tf = self.offset_tf
        dl_btn = self.dl_btn
        table_btn = self.table_btn
        out = self.out
        
        dl_btn.on_event(
            'click',
            partial(
                on_click_get_data,
                self=self
            )
        )
        table_btn.on_event(
            'click',
            partial(
                out.capture(clear_output=True)(on_click_chart),
                self=self
            )
        )
    
        super().link()
        
def on_click_get_data(widget, event, payload, self):
    widget.loading = True
    try:
        self.download()
    except Exception as e:
        widget.color = 'error'
        raise e
    else:
        widget.color = 'success'
    finally:
        widget.loading = False
    

def on_click_chart(widget, event, payload, self):
    widget.loading = True
    try:
        self.get_chart_df()
        fig = self.get_figure()
        display(fig)
        dg = make_grid(self.b_mkt)
        self.dg = dg
        display(dg)
    except Exception as e:
        widget.color = 'error'
        raise e
    else:
        widget.color = 'success'
    finally:
        widget.loading = False
    

In [40]:
self = Visual(bond_ref)
self